<a href="https://colab.research.google.com/github/427paul/ai_agent/blob/main/02_LangChain%EA%B8%B0%EC%B4%881.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **02.LangChain기초1**



## **1.환경준비**

### (1) 라이브러리

#### 1) 필요한 라이브러리 설치

In [ ]:
!pip install langchain openai langchain-openai langchain_community langchain-huggingface langchain-core huggingface_hub -q

#### 2) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import os

import openai
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

### (2) API Key 설정

* API 키 설정은 다음 둘 중 한가지를 수행해야 합니다.
    * 허깅페이스 키 : 무료, 사용 제한 있음
    * OpenAI 키 : 유료

* 허깅페이스

In [ ]:
# hf key 설정
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your key"

* openai

In [ ]:
# openai api key
os.environ["OPENAI_API_KEY"] = "your key"

## **2. Model**

* 모델도 허깅페이스 모델과 OpenAI 모델 둘 중, 키를 설정한 한가지를 사용하면 됩니다.

### (1) 모델 준비

* 허깅페이스

In [ ]:
# HuggingFace에서 해당 모델을 불러오는 엔드포인트 지정
llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="conversational")

# HuggingFace에서 가져온 모델을 그대로 쓰지 않고,
# LangChain에서 쉽게 쓰도록 감싸는(wrapper) 단계
llm1 = ChatHuggingFace(llm=llm_ep)

* OpenAI

In [ ]:
llm2 = ChatOpenAI(model = "gpt-4.1-mini")

### (2) 사용하기1

* 허깅페이스

In [ ]:
resp = llm1.invoke("세계에서 가장 큰 산은?")
print(resp.content)

* OpenAI

In [ ]:
resp = llm2.invoke("세계에서 가장 큰 산은?")
print(resp.content)

### (3) 사용하기2 : 역할 부여

* HF

In [ ]:
# 역할부여
sys_role = '당신은 애국심을 가지고 있는 건전한 대한민국 국민입니다.'
question = "독도는 어느나라 땅인가요?"

resp = llm1.invoke([SystemMessage(content = sys_role), HumanMessage(content = question)])
print(resp.content)

* OpenAI

In [ ]:
# 역할부여
sys_role = '당신은 애국심을 가지고 있는 건전한 대한민국 국민입니다.'
question = "독도는 어느나라 땅인가요?"

resp = llm2.invoke([SystemMessage(content = sys_role), HumanMessage(content = question), ])
print(resp.content)

## **3.ChatPromptTemplate**

- 시스템 메시지, 사용자 메시지, AI 메시지 등 역할(role) 구분
- 다중 메시지 기반의 프롬프트 흐름을 구성할 수 있도록 도와주는 템플릿

**메세지 종류**
- SystemMessage : AI에게 역할/성격을 지정
- HumanMessage : 사용자 질문 또는 요청
- AIMessage : AI 응답


여러 메시지를 구조화하여 대화 설계

### (1) 모델 준비

* HF

In [ ]:
llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="conversational",
                            temperature=0.7, top_p=0.9)
llm1 = ChatHuggingFace(llm=llm_ep)

* OpenAI

In [ ]:
llm2 = ChatOpenAI(model_name = 'gpt-4.1-mini',
                 temperature=0.7, top_p=0.9)

### (2) 템플릿 사용

In [ ]:
s_msg = "너는 친절하고 유머 있는 상담사야."
h_msg = "요즘 너무 지치고 의욕이 없어. 어떻게 하면 좋을까?"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", s_msg), ("human", h_msg)
    ])

* HF

In [ ]:
messages = chat_prompt.format_messages() # 실제 메시지 객체 리스트를 생성
response = llm1.invoke(messages)
print(response.content)

* OpenAI

In [ ]:
messages = chat_prompt.format_messages() # 실제 메시지 객체 리스트를 생성
response = llm2.invoke(messages)
print(response.content)

* 템플릿 열어보기

In [ ]:
chat_prompt

In [ ]:
messages

In [ ]:
response

In [ ]:
print(response.content)

### (3) 입력변수 사용

In [ ]:
s_msg = "너는 {role}야."
h_msg = "요즘 너무 지치고 의욕이 없어. 어떻게 하면 좋을까?"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", s_msg),
    ("human", h_msg),
])

print(chat_prompt)

In [ ]:
messages = chat_prompt.format_messages(role="친절한 상담사")
print(messages)

In [ ]:
resp = llm2.invoke(messages)
print(resp.content)

In [ ]:
messages = chat_prompt.format_messages(role="영화 평론가")
resp = llm2.invoke(messages)
print(resp.content)

### (4) 😀실습

* 영화 추천 템플릿 만들기
    * 입력변수 : 장르
    * 장르를 입력받아 영화 1편과 추천이유를 설명하는 템플릿을 만들고 사용해 봅시다.

## **4. OutputParser**

**Output Parser**
- LLM에서 반환된 자유형 텍스트(string)를를 우리가 원하는 형태로 가공해주는 도구

**Output Parser의 종류**
- CommaSeparatedListOutputParser : 쉼표 구분 문자열 → 리스트로 변환
- PydanticOutputParser : 텍스트 → Pydantic 모델로 파싱
- StructuredOutputParser : JSON 기반 구조화 파싱

### (1) PydanticOutputParser

In [ ]:
from pydantic import BaseModel
from langchain.output_parsers import PydanticOutputParser

#### 1) Pydantic

In [ ]:
class User(BaseModel):
    name: str
    age: int

#### 2) 출력파서로 이용

* llm의 성능에 따라 출력 파싱에 맞게 적절한  답변을 할 수도 있고, 잘못 답변해서 오류가 날 수도 있음.

In [ ]:
# 1. Pydantic 모델 정의
class BookInfo(BaseModel):
    title: str
    author: str
    year: int

# 2. 파서 생성
parser = PydanticOutputParser(pydantic_object=BookInfo)

# 3. 프롬프트 구성 (ChatPromptTemplate 사용)
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 책 추천 전문가야."),
    ("human", "좋은 책 하나만 추천해줘. 제목과 저자, 출판년도를 알려줘."),
    ("system", "{output_format}")  # 파서가 제공한 응답 형식 가이드
])

# 4. 메시지 생성
messages = prompt.format_messages(
    output_format = parser.get_format_instructions()
)

# 5. LLM 호출 및 파싱
response = llm2.invoke(messages)
book = parser.parse(response.content)

# 6. 결과 출력
print(book)

In [ ]:
messages

### (2) 😀실습

* 장르를 입력하면, 영화 제목, 감독, 주연배우, 연도를 출력하도록 합시다. (PydanticOutputParser사용)